In [1]:
%pip install -U keras-segmentation opencv-python
%tensorflow_version 1.x #TODO test with 2.x (which should be smarter un GPU/TPU usage)
import keras_segmentation as ks

Requirement already up-to-date: opencv-python in /usr/local/lib/python3.6/dist-packages (4.1.2.30)
  Created wheel for keras-segmentation: filename=keras_segmentation-0.2.0-cp36-none-any.whl size=25694 sha256=30abd0a4eab1d8bf9a253b54c087de833a8b66bdcb9e8bac6186e742de6b7ed6
  Stored in directory: /root/.cache/pip/wheels/10/43/34/8d5a4aaa8d4479380c3aef57abf90ac47aa1e062c772defa39
Successfully built keras-segmentation
`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x #TODO test with 2.x (which should be smarter un GPU/TPU usage)`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
# data has been splitted manually between train and validation sets
import sys
IN_COLAB = 'google.colab' in sys.modules
if (IN_COLAB):
    from google.colab import drive
    drive.mount('/content/gdrive')
    PATH_OF_DATA= '/content/gdrive/My Drive/Documents/EPFL/ML_Project_2/data'
else:
    PATH_OF_DATA= './data'
IMAGES_SUFFIX='images'
GD_SUFFIX='groundtruth'
TRAIN_PREFIX='train'
VALIDATION_PREFIX='val'
PREPPED='prepped'
!ls {PATH_OF_DATA}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
ls: cannot access '/content/gdrive/My': No such file or directory
ls: cannot access 'Drive/Documents/EPFL/ML_Project_2/data': No such file or directory


In [3]:
#model = ks.models.unet.vgg_unet(n_classes=2, input_height=400, input_width=400) #TODO: try to use builtin model with cropped images
from keras.models import *
from keras.layers import *
n_classes = 2
input_width = 400
input_height = input_width

img_input = Input(shape=(input_height, input_width, 3))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.2)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = Dropout(0.2)(conv5)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

out = Conv2D(n_classes, (1, 1), padding='same')(conv5)

from keras_segmentation.models.model_utils import get_segmentation_model

model = get_segmentation_model(img_input ,  out) # this would build the segmentation model





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
model.train( 
    train_images =  f'{PATH_OF_DATA}/{TRAIN_PREFIX}_{PREPPED}_{IMAGES_SUFFIX}',
    train_annotations = f'{PATH_OF_DATA}/{TRAIN_PREFIX}_{PREPPED}_{GD_SUFFIX}',
    checkpoints_path = f'{PATH_OF_DATA}/vgg_unet_1' , epochs=5000
)



Verifying train dataset


100%|██████████| 79/79 [00:53<00:00,  1.45it/s]


Dataset verified! 
Starting Epoch  0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1





512/512 [==============================] - 116s 226ms/step - loss: 0.5214 - acc: 0.7730
saved  /content/gdrive/My Drive/Documents/EPFL/ML_Project_2/data/vgg_unet_1.model.0
Finished Epoch 0
Starting Epoch  1
Epoch 1/1
512/512 [==============================] - 103s 201ms/step - loss: 0.3855 - acc: 0.8207
saved  /content/gdrive/My Drive/Documents/EPFL/ML_Project_2/data/vgg_unet_1.model.1
Finished Epoch 1
Starting Epoch  2
Epoch 1/1
512/512 [==============================] - 103s 200ms/step - loss: 0.3095 - acc: 0.8609
saved  /content/gdrive/My Drive/Documents/EPFL/ML_Project_2/data/vgg_unet_1.model.2
Finished Epoch 2
Starting Epoch  3
Epoch 1/1
512/512 [==============================] - 103s 200ms/step - loss: 0.2663 - acc: 0.8838
saved  /content/gdrive/My Drive/Documents/EPFL/ML_Project_2/data/vgg_unet_1.model.3
Finished Epoch 3
Starting Ep